# Create parameters and mechanism json files

This notebook creates the `mechanisms.json`, `parameters.json`, and `parameters_release.json` from the `pyr_mainaxon_reboot.json` file provided by Dr. Zbili.

In [ ]:
import json
from pprint import pprint
from copy import copy
import numpy as np

In [ ]:
param_file = "parameters.json"
mechanism_file = "mechanisms.json"
param_release_file = "parameters_release.json"
param_info_file = "pyr_mainaxon_reboot.json"

In [ ]:
param_info = json.load(open(param_info_file))
pprint(param_info)

In [ ]:
sections_mech_map = {
    "basal": ["basal", "somadend"], # all dendrites are "basal" and active
    "somatic": ["somatic", "somadend"],
    "axonal": ["axonal"],
    "axon_initial_segment": ["axon_initial_segment"]
    }

## Create mechanisms.json

In [ ]:
mechanisms = {}
for sec, param_secs in sections_mech_map.items():
    mechanisms[sec] = []
    for param_sec in param_secs:
        mechanisms[sec].extend(param_info["mechanisms"][param_sec]["mech"])
mechanisms["all"] = ["pas"]
pprint(mechanisms)

json.dump(mechanisms, open(mechanism_file, "w"), indent=4)

## Create parameters.json and parameters_release.json

For the `parameters_release.json`, the mean of the `bounds` is used.

In [ ]:
params = []
params_release = []

# set global params
for par in param_info["parameters"]["global"]:
    par_dict = {"param_name": par["name"],
                "type": "global",
                "value": par["val"]}
    params.append(par_dict)
    params_release.append(par_dict) 
    
# add "all" specific parameters
mech_all = param_info["mechanisms"]["all"]
for par_all in param_info["parameters"]["all"]:
    par_dict = {"param_name": par_all["name"],
                "type": "section",
                "sectionlist": "all",
                "dist_type": "uniform"}
    par_dict_release = copy(par_dict)
    # check if a mech needs to be added
    for mech in mech_all["mech"]:
        if mech in par_dict["param_name"]:
            par_dict["mech"] = mech
    # add values
    if isinstance(par_all["val"], list):
        par_dict["bounds"] = par_all["val"]
        par_dict_release["value"] = np.mean(par_all["val"])
    else: # scalar
        par_dict["value"] = par_all["val"]
        par_dict_release["value"] = par_all["val"]
    params.append(par_dict)
    params_release.append(par_dict_release)    

    
# add section specific mechanisms
for sec, param_secs in sections_mech_map.items():
    for param_sec in param_secs:
        print("\nSectionlist", param_sec)
        mechs = param_info["mechanisms"][param_sec]
        for par in param_info["parameters"][param_sec]:
            par_dict = {"param_name": par["name"],
                        "type": "section",
                        "sectionlist": sec
                       }
            if "dist" not in par:
                par_dict.update({"dist_type": "uniform"})
            else:
                par_dict.update({"dist_type": par["dist"]})
                par_dict.update({"dist": param_info["distributions"][par["dist"]]["fun"]})
                for dist_par in param_info["distributions"][par["dist"]]:
                    if dist_par != "__comment" and dist_par != "fun":
                        par_dict.update({dist_par: param_info["distributions"][par["dist"]][dist_par]})
                par_dict["type"] = "range"
            par_dict_release = copy(par_dict)
            # check if a mech needs to be added
            for mech in mechs["mech"]:
                if mech in par_dict["param_name"]:
                    par_dict["mech"] = mech
                    mech_param = par_dict["param_name"][:par_dict["param_name"].find(mech) - 1]
                    par_dict["mech_param"] = mech_param
                    
            # add values
            if isinstance(par["val"], list):
                par_dict["bounds"] = par_all["val"]
                par_dict_release["value"] = np.mean(par_all["val"])
            else: # scalar
                par_dict["value"] = par["val"]
                par_dict_release["value"] = par["val"]
                            
            params.append(par_dict)
            params_release.append(par_dict_release) 
            
json.dump(params, open(param_file, "w"), indent=4)
json.dump(params_release, open(param_release_file, "w"), indent=4)

In [ ]:
pprint(params)

In [ ]:
pprint(params_release)